In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException ,StaleElementReferenceException
from selenium.webdriver.support import expected_conditions
#time
import time
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import HTMLSession
from unidecode import unidecode
from datetime import date
from bs4 import BeautifulSoup
import requests


In [22]:
def full():

    PATH= '/usr/lib/chromium-browser/chromedriver'

    link = 'https://diaonline.supermercadosdia.com.ar/'

    s= Service(PATH)

    driver = webdriver.Chrome(service=s)

    driver.set_window_size(1366,768)

    driver.get(link)

    time.sleep(12)

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)

    buttons_a= driver.find_elements(By.CSS_SELECTOR,'div.vtex-menu-2-x-styledLinkContainer.vtex-menu-2-x-styledLinkContainer--compra-online')


    categoria=[]
    for i in buttons_a:
        #print(i.text)
        categoria.append(unidecode(i.text).lower()) #saco tildes y hago minuscula


    buttons_b= driver.find_elements(By.CSS_SELECTOR,'div.vtex-menu-2-x-styledLinkContainer.vtex-menu-2-x-styledLinkContainer--compra-online-2')

    for i in buttons_b:
        
        categoria.append(unidecode(i.text).lower()) #sacar tildes y hacerlo minuscula

    full_link=[]
    for elemento in categoria:
        if " " in elemento:
            sub=elemento.split(" ")

            substring="-".join(sub)
            if substring == 'frutas-y-verduras':
                corregido="/".join([ 'frescos','frutas-y-verduras'])
                full_link.append(link+corregido) #sumo link y la parte de la categoria
            else:

                full_link.append(link+substring) #sumo link y la parte de la categoria

        else:
            full_link.append(link+elemento)

    df = pd.DataFrame({'categoria':categoria,'links': full_link})

    return df

data=full()


today = date.today() #hold the date
d1 = today.strftime("%Y%m%d") 
data.to_csv(f'../csv/dia/{d1}dia_links.csv', encoding='utf-8', index=False) #always printing the date of the csv file


KeyboardInterrupt: 

In [2]:
df = pd.read_csv('../csv/dia/20230411dia_links.csv')

df

,categoria,links
0,especial ofertas,https://diaonline.supermercadosdia.com.ar/espe...
1,almacen,https://diaonline.supermercadosdia.com.ar/almacen
2,desayuno,https://diaonline.supermercadosdia.com.ar/desa...
3,bebidas,https://diaonline.supermercadosdia.com.ar/bebidas
4,frescos,https://diaonline.supermercadosdia.com.ar/frescos
5,frutas y verduras,https://diaonline.supermercadosdia.com.ar/fres...
6,congelados,https://diaonline.supermercadosdia.com.ar/cong...
7,perfumeria,https://diaonline.supermercadosdia.com.ar/perf...
8,limpieza,https://diaonline.supermercadosdia.com.ar/limp...
9,bebes y ninos,https://diaonline.supermercadosdia.com.ar/bebe...


In [2]:
def products(categoria,source):

    

    #r = requests.get(url)


    soup =BeautifulSoup(source,'html.parser')

    #frame = soup.find('div',class_='vtex-search-result-3-x-gallery vtex-search-result-3-x-gallery--default flex flex-row flex-wrap items-stretch bn ph1 na4 pl9-l')

    #print(frame)
    
    products =soup.find_all('div',class_='vtex-search-result-3-x-galleryItem vtex-search-result-3-x-galleryItem--normal vtex-search-result-3-x-galleryItem--default pa4')
    
    count_product=0
    product_list=[]
    for product in products:
         
        count_product=count_product+1
    #this is the count fraction to check the scrapped elements
        print(f'{count_product} / {len(products)} Elemento escrapeado')

        current_product={'market':'Dia','category': categoria,'currency':'ARG'}

        #marca producto
        brand= product.find('div',class_='vtex-product-summary-2-x-productBrandContainer').text
        current_product['marca']=brand

        #precio descripcion
        description= product.find('div',class_='vtex-product-summary-2-x-nameContainer flex items-start justify-center pv6').text
        current_product['descripcion'] =description

        #promocion
        try:
            promocion= product.find('div',class_='vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--shelf_promotions').text
            current_product['promocion'] = promocion
        except:
            current_product['promocion'] = None

        #precio producto
        precio = product.find('div',class_='vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--shelf_prices').text
    
        if precio.count('$',2):
            part = precio.split("$")          
            precio= "$"+part[1]
            precio_promocion="$"+part[2]
        else:
            precio
            
            precio_promocion= None
        
    
        
        current_product['precio_producto'] =precio
        current_product['precio_promocion'] = precio_promocion

        #precio x unidad
        precio_unidad = product.find('div',class_='vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--product-unit').text
        current_product['precio_kilo']= precio_unidad
        

        product_list.append(current_product)


    return product_list





In [46]:

def full_scrap(categoria,link):


    PATH= '/usr/lib/chromium-browser/chromedriver'

    

    s= Service(PATH)

    driver = webdriver.Chrome(service=s)

    driver.maximize_window()

    driver.get(link)

    time.sleep(12)


    if driver.current_url=='https://diaonline.supermercadosdia.com.ar/almacen':
        elemento= driver.find_elements(By.CSS_SELECTOR,'div.vtex-search-result-3-x-filter__container.vtex-search-result-3-x-filter__container--plp')
        driver.execute_script("arguments[0].scrollIntoView(0);", elemento[5])
        time.sleep(6)

        label = driver.find_elements(By.CSS_SELECTOR,"label[for*='category-2']")
        print(len(label))

#almost done
        full=[]
        for u in label:
            try:
                g=driver.find_element(By.CSS_SELECTOR,"g[transform='translate(2 2)']")
                g.click()
                time.sleep(2)
                exit= driver.find_element(By.CSS_SELECTOR,'button[title="Close survey"]')
                exit.click()
            except:
                pass
            print(u.text)
            u.click()
            time.sleep(7)

            cont=0
            prev_url = None 
            while True:
                cont+=1
                if driver.current_url == prev_url:
            # Salir del bucle si driver.current_url no ha cambiado
                    break
    
                prev_url = driver.current_url

                print(f'cantidad de hojas: {cont}') #contador
                elemento = driver.find_element(By.CSS_SELECTOR, 'div.vtex-search-result-3-x-buttonShowMore--layout')
                driver.execute_script("arguments[0].scrollIntoView(0);", elemento) 
                time.sleep(10)

            data=products(categoria,driver.page_source) #aplico la func
            full.extend(data) #extend o append?

            elemento= driver.find_elements(By.CSS_SELECTOR,'div.vtex-search-result-3-x-filter__container.vtex-search-result-3-x-filter__container--plp')
            driver.execute_script("arguments[0].scrollIntoView(0);", elemento[5])
            time.sleep(3)
            u.click()
            time.sleep(7)
            elemento= driver.find_elements(By.CSS_SELECTOR,'div.vtex-search-result-3-x-filter__container.vtex-search-result-3-x-filter__container--plp')
            driver.execute_script("arguments[0].scrollIntoView(0);", elemento[5])
            time.sleep(6)
        df= pd.DataFrame(full).replace("", None)
        

    
    else:
        cont = 0
        prev_url = None  # Almacenar el valor anterior de driver.current_url
        while True:
    
            if driver.current_url == prev_url:
            # Salir del bucle si driver.current_url no ha cambiado
                break
    
            prev_url = driver.current_url  # Actualizar el valor anterior de driver.current_url
    
            cont=cont+1
            print(f'cantidad de hojas: {cont}')
            elemento = driver.find_element(By.CSS_SELECTOR, 'div.vtex-search-result-3-x-buttonShowMore--layout')
            driver.execute_script("arguments[0].scrollIntoView(0);", elemento)
            time.sleep(10)
    

    
        data=products(categoria,driver.page_source)

        df=pd.DataFrame(data).replace("",None)
       

    driver.quit()

    return df

In [35]:

PATH= '/usr/lib/chromium-browser/chromedriver'

    

s= Service(PATH)

driver = webdriver.Chrome(service=s)

driver.maximize_window()

driver.get('https://diaonline.supermercadosdia.com.ar/almacen')

time.sleep(12)

elemento= driver.find_elements(By.CSS_SELECTOR,'div.vtex-search-result-3-x-filter__container.vtex-search-result-3-x-filter__container--plp')
driver.execute_script("arguments[0].scrollIntoView(0);", elemento[5])
time.sleep(6)

label = driver.find_elements(By.CSS_SELECTOR,"label[for*='category-2']")
print(len(label))

#almost done
for u in label:
    try:
        g=driver.find_element(By.CSS_SELECTOR,"g[transform='translate(2 2)']")
        g.click()
        time.sleep(2)
        exit= driver.find_element(By.CSS_SELECTOR,'button[title="Close survey"]')
        exit.click()
    except:
        pass
    print(u.text)
    u.click()
    time.sleep(7)

    elemento= driver.find_elements(By.CSS_SELECTOR,'div.vtex-search-result-3-x-filter__container.vtex-search-result-3-x-filter__container--plp')
    driver.execute_script("arguments[0].scrollIntoView(0);", elemento[5])
    time.sleep(3)
    u.click()
    time.sleep(7)
    elemento= driver.find_elements(By.CSS_SELECTOR,'div.vtex-search-result-3-x-filter__container.vtex-search-result-3-x-filter__container--plp')
    driver.execute_script("arguments[0].scrollIntoView(0);", elemento[5])
    time.sleep(6)



10
Aceites y Aderezos
Arroz y Legumbres
Comidas listas
Conservas
Golosinas y Alfajores
Harinas
Panadería
Pastas Secas
Picadas
Repostería


In [37]:
main_data= pd.read_csv('../csv/dia/20230411dia_links.csv')

In [43]:
main_data= main_data.loc[1:]

In [47]:
main=[full_scrap(i,u) for i,u in zip(main_data['categoria'],main_data['links'])]

10
Aceites y Aderezos
cantidad de hojas: 1
cantidad de hojas: 2
cantidad de hojas: 3
cantidad de hojas: 4
cantidad de hojas: 5
cantidad de hojas: 6
cantidad de hojas: 7
cantidad de hojas: 8
cantidad de hojas: 9
cantidad de hojas: 10
cantidad de hojas: 11
1 / 165 Elemento escrapeado
2 / 165 Elemento escrapeado
3 / 165 Elemento escrapeado
4 / 165 Elemento escrapeado
5 / 165 Elemento escrapeado
6 / 165 Elemento escrapeado
7 / 165 Elemento escrapeado
8 / 165 Elemento escrapeado
9 / 165 Elemento escrapeado
10 / 165 Elemento escrapeado
11 / 165 Elemento escrapeado
12 / 165 Elemento escrapeado
13 / 165 Elemento escrapeado
14 / 165 Elemento escrapeado
15 / 165 Elemento escrapeado
16 / 165 Elemento escrapeado
17 / 165 Elemento escrapeado
18 / 165 Elemento escrapeado
19 / 165 Elemento escrapeado
20 / 165 Elemento escrapeado
21 / 165 Elemento escrapeado
22 / 165 Elemento escrapeado
23 / 165 Elemento escrapeado
24 / 165 Elemento escrapeado
25 / 165 Elemento escrapeado
26 / 165 Elemento escrapeado


In [48]:
main

[     market category currency      marca  \
 0       Dia  almacen      ARG   LEGITIMO   
 1       Dia  almacen      ARG  SIN MARCA   
 2       Dia  almacen      ARG     NATURA   
 3       Dia  almacen      ARG     NATURA   
 4       Dia  almacen      ARG    CELUSAL   
 ...     ...      ...      ...        ...   
 1158    Dia  almacen      ARG        DIA   
 1159    Dia  almacen      ARG        DIA   
 1160    Dia  almacen      ARG        DIA   
 1161    Dia  almacen      ARG        DIA   
 1162    Dia  almacen      ARG        JA!   
 
                                             descripcion promocion  \
 0                   Aceite de Girasol Legítimo 1,5 Lt.       None   
 1                   Aceite de Girasol Legítimo 900 Ml.       None   
 2                    Aceite de Girasol Natura 1,5 Lts.       None   
 3                        Aceite Girasol Natural 0,9 L.       None   
 4                  Sal Fina en Paquete Celusal 500 Gr.        25%   
 ...                                  

In [49]:
dfs=[pd.DataFrame(i) for i in main]
df=pd.concat(dfs)
df

,market,category,currency,marca,descripcion,promocion,precio_producto,precio_promocion,precio_kilo
0,Dia,almacen,ARG,LEGITIMO,"Aceite de Girasol Legítimo 1,5 Lt.",None,"$ 446,50",None,297.67LT
1,Dia,almacen,ARG,SIN MARCA,Aceite de Girasol Legítimo 900 Ml.,None,"$ 279,50",None,310.56LT
2,Dia,almacen,ARG,NATURA,"Aceite de Girasol Natura 1,5 Lts.",None,"$ 496,20",None,330.80LT
3,Dia,almacen,ARG,NATURA,"Aceite Girasol Natural 0,9 L.",None,"$ 310,60",None,345.11LT
4,Dia,almacen,ARG,CELUSAL,Sal Fina en Paquete Celusal 500 Gr.,25%,"$ 119,00","$ 158,00",238.00KG
...,...,...,...,...,...,...,...,...,...
126,Dia,electro hogar,ARG,ELECTROLUX,Lavarropas Carga Frontal Electrolux ELAF08W 8kg,8%,"$ 191.799,00","$ 208.799,00",None
127,Dia,electro hogar,ARG,ELECTROLUX,Secarropas Electrolux Eseb6 6kg,8%,"$ 141.399,00","$ 153.299,00",None
128,Dia,electro hogar,ARG,WHIRLPOOL,Accesorio KitchenAid Gancho amasador de aluminio,5%,"$ 11.199,00","$ 11.799,00",None
129,Dia,electro hogar,ARG,WHIRLPOOL,Accesorio Escudo vertedor Heavy Duty,5%,"$ 8.999,00","$ 9.499,00",None


In [50]:
df[df['category']=='almacen']

,market,category,currency,marca,descripcion,promocion,precio_producto,precio_promocion,precio_kilo
0,Dia,almacen,ARG,LEGITIMO,"Aceite de Girasol Legítimo 1,5 Lt.",None,"$ 446,50",None,297.67LT
1,Dia,almacen,ARG,SIN MARCA,Aceite de Girasol Legítimo 900 Ml.,None,"$ 279,50",None,310.56LT
2,Dia,almacen,ARG,NATURA,"Aceite de Girasol Natura 1,5 Lts.",None,"$ 496,20",None,330.80LT
3,Dia,almacen,ARG,NATURA,"Aceite Girasol Natural 0,9 L.",None,"$ 310,60",None,345.11LT
4,Dia,almacen,ARG,CELUSAL,Sal Fina en Paquete Celusal 500 Gr.,25%,"$ 119,00","$ 158,00",238.00KG
...,...,...,...,...,...,...,...,...,...
1158,Dia,almacen,ARG,DIA,Bizcochuelo Rectangular DIA 3 Capas Vainilla 7...,None,"$ 863,00",None,1150.67KG
1159,Dia,almacen,ARG,DIA,Bizcochuelo en Polvo DIA Vainilla 540 Gr.,None,"$ 409,97",None,759.20KG
1160,Dia,almacen,ARG,DIA,Gelatina DIA Frutilla 110 Gr.,None,"$ 73,30",None,666.36KG
1161,Dia,almacen,ARG,DIA,Bizcochuelo en Polvo DIA Chocolate 540 Gr.,None,"$ 409,97",None,759.20KG


In [51]:

today = date.today()
d1 = today.strftime("%Y%m%d")
#df_products.to_csv(f'.mercado_project/csv/{d1}_vea_products.csv', encoding='utf-8', index=False)
#df_products.to_csv(f'./Desktop/programacion/mercado_project/csv/{d1}_vea_products.csv'.format(d1), encoding='utf-8', index=False)
df.to_csv(f'../csv/dia/{d1}_dia_products.csv', encoding='utf-8', index=False)